In [1]:
import openeo
import pandas as pd

In [42]:
# connect to openeo
connection = openeo.connect("https://openeocloud.vito.be/openeo/1.0.0")
connection.list_collection_ids()[:5]

['MAPEO_WATER_TUR_V1',
 'COP_DEM_EU_25M',
 'ESA_WORLDCEREAL_ACTIVECROPLAND',
 'ESA_WORLDCEREAL_IRRIGATION',
 'ESA_WORLDCEREAL_TEMPORARYCROPS']

In [43]:
# Authenticate with OIDC authentication"
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeocloud.vito.be/openeo/1.0.0' with OidcBearerAuth>

In [46]:
# see https://docs.openeo.cloud/data-collections/
# and https://hub.openeo.org/
datacube = connection.load_collection(collection_id='SENTINEL1_GRD',
                                      properties=['sar:instrument_mode' == 2, #IW
                                                  'sar:satellite:orbit_state' == 2, #descending
                                                  'sar:resolution' == 1, #10 range, 10 azimuth; max ability to distinguish 2 close scatters
                                                  'sar:range_pixel_spacing' == 1, #10m pixel size
                                                  'data_type' == 'gamma0',
                                                  'bands' == ['VH', 'HH', 'HV', 'VV'],
                                                  'orthorectify' == 'true'])

In [47]:
def divide_bounds(min_y, max_y, n):
    
    north_lats = []
    south_lats = []
    step = (max_y - min_y) / n
    
    for i in range(n):
        north = max_y - i * step
        south = north - step
        north_lats.append(round(north, 1))
        south_lats.append(round(south, 1))
        
    return north_lats, south_lats

In [48]:
# set min and max lat and number of steps
min_y = 52
max_y = 72
min_x = -169.1
max_x = -139.8

# do it
norths, souths = divide_bounds(min_y, max_y, 8)
easts, wests = divide_bounds(min_x, max_x, 8)

In [49]:
print(norths, souths, wests, easts)

[72.0, 69.5, 67.0, 64.5, 62.0, 59.5, 57.0, 54.5] [69.5, 67.0, 64.5, 62.0, 59.5, 57.0, 54.5, 52.0] [-143.5, -147.1, -150.8, -154.4, -158.1, -161.8, -165.4, -169.1] [-139.8, -143.5, -147.1, -150.8, -154.4, -158.1, -161.8, -165.4]


In [50]:
i = 0

# filter by location, date, and bands
datacube = datacube.filter_bbox(west=wests[i], south=souths[i], east=easts[i], north=norths[i])
datacube = datacube.filter_temporal(start_date="2019-06-01", end_date="2019-08-31")
datacube = datacube.filter_bands(["VV", "VH"])

In [51]:
# mask and create median
masked_datacube = datacube.apply(lambda x:x <= -30.0)
datacube = datacube.mask(masked_datacube)
med_summer = datacube.median_time()

In [52]:
# backscatter
med_summer = med_summer.sar_backscatter(coefficient='gamma0-ellipsoid')

In [53]:
med_summer.print_json()

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "id": "SENTINEL1_GRD",
        "spatial_extent": null,
        "temporal_extent": null
      }
    },
    "filterbbox1": {
      "process_id": "filter_bbox",
      "arguments": {
        "data": {
          "from_node": "loadcollection1"
        },
        "extent": {
          "west": -143.5,
          "east": -139.8,
          "north": 72.0,
          "south": 69.5
        }
      }
    },
    "filtertemporal1": {
      "process_id": "filter_temporal",
      "arguments": {
        "data": {
          "from_node": "filterbbox1"
        },
        "extent": [
          "2019-06-01",
          "2019-08-31"
        ]
      }
    },
    "filterbands1": {
      "process_id": "filter_bands",
      "arguments": {
        "bands": [
          "VV",
          "VH"
        ],
        "data": {
          "from_node": "filtertemporal1"
        }
      }
    },
    "apply1": {
      

In [54]:
# create result
result = med_summer.save_result(format="GTiff")
result

In [55]:
result.download()

OpenEoApiError: [403] PermissionsInsufficient: Proper enrollment in openEO Platform virtual organization is required. (ref: r-23122076871543b2b55d633b8207b1dd)

In [1]:
import asf_search as asf
import geopandas as gpd
from shapely.geometry import Polygon, box
from datetime import date
import pandas as pd
import numpy as np
import requests
import os

In [2]:
def divide_bounds(min_y, max_y, n):
    
    north_lats = []
    south_lats = []
    step = (max_y - min_y) / n
    
    for i in range(n):
        north = max_y - i * step
        south = north - step
        north_lats.append(round(north, 1))
        south_lats.append(round(south, 1))
        
    return north_lats, south_lats

In [3]:
# overriding requests.Session.rebuild_auth to mantain headers when redirected
class SessionWithHeaderRedirection(requests.Session):

    AUTH_HOST = 'urs.earthdata.nasa.gov'
    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

   # Overrides from the library to keep headers when redirected to or from
   # the NASA auth host.
    def rebuild_auth(self, prepared_request, response):
        
        headers = prepared_request.headers
        url = prepared_request.url

        if 'Authorization' in headers:

            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)

            if ((original_parsed.hostname != redirect_parsed.hostname) and
                redirect_parsed.hostname != self.AUTH_HOST and 
                original_parsed.hostname != self.AUTH_HOST):
                del headers['Authorization']

        return

In [4]:
# set min and max lat and number of steps
min_y = 52
max_y = 72
min_x = -169.1
max_x = -139.8

# do it
norths, souths = divide_bounds(min_y, max_y, 8)
easts, wests = divide_bounds(min_x, max_x, 8)

In [5]:
nw = pd.DataFrame({'w':wests, 'n':norths})
ne = pd.DataFrame({'e':easts, 'n':norths})
sw = pd.DataFrame({'w':wests, 's':souths})
se = pd.DataFrame({'e':easts, 's':souths})

In [6]:
nw['nw'] = nw.values.tolist()
ne['ne'] = ne.values.tolist()
sw['sw'] = sw.values.tolist()
se['se'] = se.values.tolist()

In [7]:
all_urls = []
for i in range(1,len(nw)):
    
    # get bounds
    poly = Polygon([nw['nw'][i], ne['ne'][i], se['se'][i], sw['sw'][i]])
    bounds = poly.bounds
    gdf_bounds = gpd.GeoSeries([box(*bounds)], crs='EPSG:4326')
    wkt_aoi = gdf_bounds.to_wkt().values.tolist()[0]
    
    # initial query through asf
    asf.CMR_TIMEOUT = 60
    results = asf.search(platform=asf.PLATFORM.SENTINEL1,
                         processingLevel=[asf.PRODUCT_TYPE.GRD_HD, 
                                          asf.PRODUCT_TYPE.GRD_FD,
                                          asf.PRODUCT_TYPE.GRD_HS,
                                          asf.PRODUCT_TYPE.GRD_MD,
                                          asf.PRODUCT_TYPE.GRD_MS],
                         start=date(2019, 6, 1),
                         end=date(2019, 8, 31),
                         intersectsWith=wkt_aoi)

    # metadata to dictionary
    metadata = results.geojson()
    
    # get flight direction metadata
    direction = []
    for f in range(len(metadata['features'])):
        flightDirection = metadata['features'][f]['properties']['flightDirection']
        if flightDirection:
            direction.append(flightDirection)

    # determine flight path filter
    direction_counts = np.unique(direction, return_counts=True)
    if direction_counts[1][0] > direction_counts[1][1]:
        flight_path = direction_counts[0][0]
    else:
        flight_path = direction_counts[0][1]

    # filter by metadata properties
    asf.CMR_TIMEOUT = 60
    results = asf.search(platform=asf.PLATFORM.SENTINEL1,
                         processingLevel=[asf.PRODUCT_TYPE.GRD_HD, 
                                          asf.PRODUCT_TYPE.GRD_FD,
                                          asf.PRODUCT_TYPE.GRD_HS,
                                          asf.PRODUCT_TYPE.GRD_MD,
                                          asf.PRODUCT_TYPE.GRD_MS],
                         start=date(2019, 6, 1),
                         end=date(2019, 8, 31),
                         intersectsWith=wkt_aoi,
                         beamMode='IW',
                         flightDirection=flight_path)
    metadata = results.geojson()
    
    # get urls
    for f in range(len(metadata['features'])):    
        url = metadata['features'][f]['properties']['url']
        #if os.path.isfile(outpath) == False:
        all_urls.append(url)

In [8]:
EARTHDATA_USER='msteckler98'
EARTHDATA_PASS='E@rthd@t@1998!'
session = SessionWithHeaderRedirection(EARTHDATA_USER, EARTHDATA_PASS)

In [10]:
for url in all_urls:
    print(url)

https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190829T155510_20190829T155535_028786_0342B3_16CC.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190829T155441_20190829T155510_028786_0342B3_C79F.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190827T161126_20190827T161151_028757_0341B4_91B5.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190827T161101_20190827T161126_028757_0341B4_9913.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190827T161032_20190827T161101_028757_0341B4_0A1F.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190825T162747_20190825T162812_028728_0340B3_C627.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190825T162722_20190825T162747_028728_0340B3_630D.zip
https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20190825T162653_20190825T162722_028728_0340B3_271F.zip
https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_IW_GRDH_1SDV_20190821T161013_20190821T1610

In [10]:
txt = '/mnt/poseidon/remotesensing/arctic/scripts/snap_esa/Sentinel1/s1_alaska_https.txt'
for url in all_urls:
    with open(txt, 'a') as file:
        file.write(f'{url}\n')

In [41]:
# authenticate with earthdata credentials
# USERNAME = 'msteckler98'
# PASSWORD = 'E@rthd@t@1998!'
# session = asf.ASFSession().auth_with_creds(USERNAME, PASSWORD)